In [ ]:
# Дан файл с некоторыми данными. Формат файла – произвольный. 
# Спроектируйте на его основе и создайте таблицу в базе данных (SQLite). 
# Считайте данные из файла и запишите их в созданную таблицу. 
# Реализуйте и выполните следующие запросы:
#    вывод первых (VAR+10) отсортированных по произвольному числовому полю строк из таблицы в файл формата json;
#    вывод (сумму, мин, макс, среднее) по произвольному числовому полю;
#    вывод частоты встречаемости для категориального поля;
#    вывод первых (VAR+10) отфильтрованных по произвольному предикату отсортированных по произвольному числовому полю строк 
#    из таблицы в файл формате json.

In [1]:
import pandas as pd
import sqlite3
import pickle
import json

In [2]:
with open(b'task_1_var_17_item.pkl', 'rb') as f:
    data = pickle.load(f)
data[0:3]    

[{'id': 829265,
  'name': 'Тенета 54',
  'street': 'Петербургская улица 92',
  'city': 'Луарка',
  'zipcode': 751186,
  'floors': 5,
  'year': 1730,
  'parking': True,
  'prob_price': 525577561,
  'views': 56406},
 {'id': 715475,
  'name': 'Водопровод 24',
  'street': 'Юбилейная улица 62',
  'city': 'Афины',
  'zipcode': 735823,
  'floors': 4,
  'year': 1644,
  'parking': True,
  'prob_price': 996274599,
  'views': 36862},
 {'id': 899425,
  'name': 'Музей 86',
  'street': 'Вроцлавская улица 92',
  'city': 'Картахена',
  'zipcode': 109371,
  'floors': 2,
  'year': 1860,
  'parking': True,
  'prob_price': 634005149,
  'views': 62284}]

In [3]:
# Создание таблицы в базе данных
def create_table():
    conn = sqlite3.connect('database.db')
    conn.text_factory = str
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS properties
                  (id INT PRIMARY KEY,
                  name TEXT,
                  street TEXT,
                  city TEXT,
                  zipcode INT,
                  floors INT,
                  year INT,
                  parking BOOLEAN,
                  prob_price INT,
                  views INT)''')
    conn.commit()
    conn.close()

In [4]:
# Чтение данных из файла в таблицу
def insert_data():
    conn = sqlite3.connect('database.db')
    c = conn.cursor()
    # Чтение данных из файла task_1_var_17_item.pkl
    with open('task_1_var_17_item.pkl', 'rb') as file:
        data = pickle.load(file)
    # Запись данных в таблицу
    for item in data:
        c.execute("INSERT OR IGNORE INTO properties VALUES (:id, :name, :street, :city, :zipcode, :floors, :year, :parking, :prob_price, :views)",
                   {'id': item['id'], 
                    'name': item['name'], 
                    'street': item['street'], 
                    'city': item['city'], 
                    'zipcode': item['zipcode'], 
                    'floors': item['floors'], 
                    'year': item['year'], 
                    'parking': item['parking'], 
                    'prob_price': item['prob_price'], 
                    'views': item['views']})
    # Сохранение изменений
    conn.commit()
    conn.close()    

In [5]:
# Вывод первых 27 отсортированных по числовому полю ['year'] в файл json
def sort_by_year():
    conn = sqlite3.connect('database.db')
    c = conn.cursor()
    c.execute("SELECT * FROM properties ORDER BY year LIMIT 27")
    result = c.fetchall()
    with open('sorted_properties_1.json', 'w', encoding='utf-8') as file:
        json.dump(result, file, ensure_ascii=False)

    conn.close()

In [6]:
# Вывод суммы, минимума, максимума и среднего значения по полю ['floors']
def floors_statistics():
    conn = sqlite3.connect('database.db')
    c = conn.cursor()

    c.execute("SELECT SUM(floors), MIN(floors), MAX(floors), AVG(floors) FROM properties")
    result = c.fetchone()

    conn.close()

    return result

In [7]:
# Вывод частоты встречаемости для категориального поля ['city']
def city_frequency():
    conn = sqlite3.connect('database.db')
    c = conn.cursor()

    c.execute("SELECT city, COUNT(*) FROM properties GROUP BY city")
    result = c.fetchall()

    conn.close()

    return result

In [8]:
# Вывод первых 27 отфильтрованных по произвольному предикату и отсортированных по числовому полю ['views'] в файл json
def filter_and_sort_by_views():
    conn = sqlite3.connect('database.db')
    c = conn.cursor()

    c.execute("SELECT * FROM properties WHERE views > 1000 ORDER BY views LIMIT 27")
    result = c.fetchall()

    with open('filtered_properties_1.json', 'w', encoding='utf-8') as file:
        json.dump(result, file, ensure_ascii=False)

    conn.close()

In [9]:
# Создание таблицы и запись данных
create_table()
insert_data()

In [10]:
# Вывод первых 27 отсортированных по числовому полю ['year'] в файл json
sort_by_year()

In [11]:
# Вывод статистики по полю ['floors']
floors_stats = floors_statistics()
floors_sum, floors_min, floors_max, floors_avg = floors_stats
floors_stats

(381, 1, 10, 5.521739130434782)

In [12]:
# Вывод частоты встречаемости для категориального поля ['city']
city_freq = city_frequency()
city_freq

[('Авилес', 2),
 ('Аликанте', 1),
 ('Алма-Ата', 2),
 ('Алькала-де-Энарес', 1),
 ('Артейхо', 2),
 ('Афины', 1),
 ('Баку', 1),
 ('Барселона', 2),
 ('Бишкек', 2),
 ('Бланес', 1),
 ('Будапешт', 1),
 ('Бургос', 1),
 ('Вальядолид', 2),
 ('Варшава', 2),
 ('Вильявисиоса', 1),
 ('Вроцлав', 1),
 ('Гранада', 2),
 ('Душанбе', 1),
 ('Кадакес', 1),
 ('Картахена', 2),
 ('Касерес', 2),
 ('Киев', 1),
 ('Куэнка', 3),
 ('Ла-Корунья', 1),
 ('Луарка', 2),
 ('Льейда', 1),
 ('Любляна', 2),
 ('Мартоса', 2),
 ('Мерида', 1),
 ('Мурсия', 1),
 ('Осера', 2),
 ('Пласенсия', 1),
 ('Подгорица', 1),
 ('Прага', 1),
 ('Самора', 2),
 ('Сан-Себастьян', 2),
 ('Санкт-Петербург', 1),
 ('Сантьяго-де-Компостела', 1),
 ('Санхенхо', 1),
 ('Сеговия', 1),
 ('Семана', 1),
 ('Сьюдад-Реаль', 1),
 ('Тарраса', 2),
 ('Тбилиси', 1),
 ('Хельсинки', 3),
 ('Хихон', 3)]

In [13]:
# Вывод первых 27 отфильтрованных по произвольному предикату и отсортированных по числовому полю ['views'] в файл json
filter_and_sort_by_views()